In [2]:
import json
import pandas as pd
import gf_utils
from gf_utils.stc_data import get_stc_data
from pathlib import Path
from datetime import datetime
from collections import defaultdict

log_dir = Path('logs/equip')
output_file = Path('equip.csv')
full_record_df = pd.read_csv(output_file)
game_data = get_stc_data(Path(gf_utils.__file__).resolve().parent/'../../data/ch/formatted/json')

In [5]:
from rich.table import Table
from rich.console import Console
from rich.style import NULL_STYLE
from datetime import datetime
td = int(datetime(2022,11,18,10).timestamp())
console = Console(force_terminal=True,force_jupyter=False,no_color=True,highlight=False)
for formula in game_data['recommended_formula'].values():
    # print(formula)
    if formula['develop_type']!=3:
        continue
    mp,ammo,mre,part = formula['mp'],formula['ammo'],formula['mre'],formula['part']
    common_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==1 and trust==True and dev_time>@td')['equip_id'].value_counts().to_frame('count')
    common_dev_df['chance'] = common_dev_df['count'].apply(lambda x: round(x/common_dev_df['count'].sum()*100,3))
    special_dev_df = full_record_df.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and item_num==0 and trust==True and dev_time>@td')['equip_id'].value_counts().to_frame('count')
    special_dev_df['chance'] = special_dev_df['count'].apply(lambda x: round(x/special_dev_df['count'].sum()*100,3))
    dev_df = common_dev_df.join(special_dev_df,rsuffix='_special',how='outer').fillna(0).reset_index()
    dev_df['count_special'] = dev_df['count_special'].astype(int)
    for key in ['name','rank']:
        dev_df[key] = dev_df['index'].apply(lambda x: game_data['equip'][x][key])
    footer = f"Count: common {dev_df['count'].sum()}, special {dev_df['count_special'].sum()}\n"
    dev_df = dev_df.query('rank==5').sort_values('chance',ascending=False)
    footer += f"Rank5: common {dev_df['chance'].sum():.3f}, special {dev_df['chance_special'].sum():.3f}"
    table = Table('           name','common','special',header_style=NULL_STYLE, title_style=NULL_STYLE, style=NULL_STYLE, caption=footer, caption_justify='left', caption_style=NULL_STYLE)
    for col in table.columns:
        col.justify='right'
    table.title = f"Formula: {mp} {ammo} {mre} {part}"
    for idx, record in dev_df.iterrows():
        table.add_row(record['name'],*[f'{record[k]:.3f}' for k in ['chance','chance_special']])
    console.print(table)
    console.print()

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│     VFL 6-24X56 │  1.984 │   1.606 │
│         EOT 518 │  1.984 │   0.803 │
│        ITI MARS │  0.794 │   0.201 │
│       AC4消音器 │  0.397 │   0.000 │
│      SF武器灯改 │  0.397 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 252, special 498        
Rank5: common 5.556, special 2.610    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────┐
│            name │ common │ special │
├─────────────────┼────────┼─────────┤
│     VFL 6-24X56 │  1.984 │   1.606 │
│         EOT 518 │  1.984 │   0.803 │
│        ITI MARS │  0.794 │   0.201 │
│       AC4消音器 │  0.397 │   0.000 │
│      SF武器灯改 │  0.397 │   0.000 │
└─────────────────┴────────┴─────────┘
Count: common 252, special 498        
Rank5: common 5.556, special 2.610    

        Formula: 140 10 110 10        
┌─────────────────┬────────┬─────────

In [3]:
output_file = Path('gun_nm.csv')
full_record_df = pd.read_csv(output_file)
game_data = get_stc_data(Path(gf_utils.__file__).resolve().parent/'../../data/ch/formatted/json')

In [4]:
ts1 = datetime(2022,10,28,10).timestamp()
ts2 = datetime(2022,12,9,10).timestamp()
ts3 = datetime(2022,12,10).timestamp()
df1 = full_record_df.query('@ts1<=dev_time and dev_time<=@ts2')
df2 = full_record_df.query('@ts2<=dev_time and dev_time<=@ts3')
df3 = full_record_df.query('@ts3<=dev_time')
for i in range(2,6):
    print(df1[f'trust_{i}'].sum())
    print(df2[f'trust_{i}'].sum())
    print(df3[f'trust_{i}'].sum())

86215
5004
8620
35892
3361
6533
17361
2507
6297
4895
696
2451


In [61]:
cfg = [
    [130,130,130,30],
    [400,400,100,200],
    [400,100,400,200],
    [100,400,400,200]
]
for mp,ammo,mre,part in cfg:
    print(mp,ammo,mre,part)
    df_t2 = df3.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_2==True')
    len2 = len(df_t2)
    vc2 = df_t2['gun_id'].value_counts(normalize=True)
    df_t5 = df3.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_5==True')
    len5 = len(df_t5)
    vc5 = df_t5['gun_id'].value_counts(normalize=True)
    print(len2, len5)
    for i in range(375,379):
        if i in vc2:
            print(i, vc2.get(i,0), vc5.get(i,0))

130 130 130 30
1625 328
375 0.005538461538461538 0.14939024390243902
400 400 100 200
1942 457
376 0.003089598352214212 0.0787746170678337
400 100 400 200
1015 433
377 0.0049261083743842365 0.07621247113163972
378 0.0009852216748768472 0.013856812933025405
100 400 400 200
1486 610
378 0.004037685060565276 0.06229508196721312


## 出率
公式 | 人形 | 五星内出率 | 总出率 | 备注
--|--|--|--|--
1110 | COLT | 14.94%(328) | 0.5538%(1625)  
4412 | ERMA | 7.88%(457) | 0.3090%(1942) | 五星内歪SCARL 2.19%
4142 | SCARH | 7.62%(433) | 0.4926%(1015) | 五星内歪SCARH 1.39%
1442 | SCARL | 6.23%(610) | 0.4038%(1486) 



In [63]:
cfg = [
    [130,130,130,30],
    [400,400,100,200],
    [400,100,400,200],
    [100,400,400,200]
]
for mp,ammo,mre,part in cfg:
    print(mp,ammo,mre,part)
    df_t2 = df2.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_2==True')
    len2 = len(df_t2)
    vc2 = df_t2['gun_id'].value_counts(normalize=True)
    df_t5 = df2.query('mp==@mp and ammo==@ammo and mre==@mre and part==@part and trust_5==True')
    len5 = len(df_t5)
    vc5 = df_t5['gun_id'].value_counts(normalize=True)
    print(len2, len5)
    for i in range(375,379):
        if i in vc2 or i in vc5:
            print(i, vc2.get(i,0), vc5.get(i,0))

130 130 130 30
1308 132
375 0.0007645259938837921 0.022727272727272728
400 400 100 200
578 89
376 0.0017301038062283738 0.0449438202247191
400 100 400 200
568 107
377 0 0.037383177570093455
100 400 400 200
791 155
378 0.0025284450063211127 0.025806451612903226


## 出率（UP前）
公式 | 人形 | 五星内出率 | 总出率 | 备注
--|--|--|--|--
1110 | COLT | 2.27%(132) | 0.0765%(1308)  
4412 | ERMA | 4.49%(89) | 0.1730%(578) 
4142 | SCARH | 3.74%(107) | 0(568) | 就是0
1442 | SCARL | 2.58%(155) | 0.2528%(791) 



In [57]:
vc2 = df_t2['gun_id'].value_counts()
vc2.keys()

Int64Index([ 68,  74,  63,  71, 107,  58,  61,  70, 133,  17,  24,  93,  94,
            105, 288,  31,  92,  27,  33,  21, 349, 262, 330,  32,  60,  69,
             26, 237,  25,  18, 118,  66, 216, 171,  72, 236, 214, 130, 356,
             62,  64, 378, 338, 122, 106,  65,  16, 313, 365, 206, 290,  22,
            243, 215, 205, 306, 196, 172, 287, 181, 194, 129, 293],
           dtype='int64')

In [59]:
vc2.get(377,0)

0